# Class and Static Methods

## First a quick review

In [3]:
class Example:   
    # a function defined within the class that does not include a reference parameter 
    # can not access class attributes implicity
    def __init__(self):
        pass
    
    def square(self, n):
        return n ** 2
                
    def square_cube(sq, cube):
        return (sq ** 2, cube ** 3)
    
    print(f"Called from within the class {square_cube(3, 2)}")  
    

Called from within the class (9, 8)


- Which of the functions in Example can be called as a bound method?
- Which of the functions is just a "ordinary" function?

### Call the function square_cube(sq, cube) from outside of the class

In [4]:
print(f"Called from outside the class by referencing the containing class name:\
      {square_cube(3, 2)}")

NameError: name 'square_cube' is not defined

In [5]:
print(f"Called from outside the class by referencing the containing class name: \
{Example.square_cube(3, 2)}")

Called from outside the class by referencing the containing class name: (9, 8)


### Create an instance of Example and call square_cube() as a method

In [6]:
ex = Example()
print(f"The value returned when calling ex.square_cube(2, 3): {ex.square_cube(2, 3)}")

TypeError: square_cube() takes 2 positional arguments but 3 were given

In [7]:
print(f"The value returned when calling ex.square(5): {ex.square(5)}")

The value returned when calling ex.square(5): 25


## Now on to Class and Static Methods

- When do you want to use a class method?
 - When you want a method that can be called from the class and from an instance of an class **and**
 - The method must be able to access class attributes
- When do you want to use a static method?
 - When you want a method that can be called from the class and from an instance of an class **and**
 - The method **must not** be able to access class attributes

### First an example of using class and static methods.

In [8]:
# Python program to demonstrate use of class and static methods
from datetime import date
  
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
      
    # a class method to create a Person object by birth year.
    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)
      
    # a static method to check if a Person is adult or not.
    @staticmethod
    def is_adult(age):
        return age >= 18
    

person1 = Person('Ann', 21)
person2 = Person.from_birth_year('Carol', 2008)
  
print (f"person1 is {person1.age}")
print (f"person2 is {person2.age}")
  
# print the result
print(f"{Person.is_adult(17)}")

person1 is 21
person2 is 14
False


Why is the @classmethod `decorator` necessary?

#### As we saw, when we define a function inside a class, whether it behaves as a function or a method depends on `whether` it is accessed from within the class, bound to the class, or bound to the instance. 

In [10]:
class Person:
    def hello(arg='default'):
        print(f'Hello, with arg={arg}')
    hello()

Hello, with arg=default


Let's call it from within the class

In [11]:
class Person:
    @classmethod
    def hello(arg='default'):
        print(f'Hello, with arg={arg}')

    hello()        

TypeError: 'classmethod' object is not callable

If we call `hello` from by referencing the class:

In [12]:
Person.hello()

Hello, with arg=default


You'll notice that `hello` was called without any arguments, in fact, `hello` is a regular function:

In [13]:
Person.hello

<function __main__.Person.hello(arg='default')>

But if we call `hello` from an instance, things are different:

In [14]:
p = Person()
print(hex(id(p)))

0x1561d285bb0


In [15]:
p.hello

<bound method Person.hello of <__main__.Person object at 0x000001561D285BB0>>

In [16]:
p.hello()

Hello, with arg=<__main__.Person object at 0x000001561D285BB0>


In [17]:
hex(id(Person))

'0x1561c75ee60'

In [18]:
Person.hello()

Hello, with arg=default


Where did the value of "with arg=" come from

Remember when we call the function bound to an instance, the first argument is passed implicitly by Python, we usually call it `self`.

And as you can see the instance `p` was passed as an argument to `hello`.   
- is there a difference between that saying `p` was passed and saying that a reference to `p` is passed?

Sometimes however, we define functions in a class that do not interact with the instance itself, but may need something from the class. In those cases, we want the class to be passed to the function as an argument, whether it is called from the class or from an instance of the class.

These are called **class methods**. You should note that the behavior needs to be different - we don't want the instance to be passed to the function when called from an instance, we want the **class** to be passed to it. In addition, when called from the class, we **also** want the class to be passed to it (this is similar to `static` methods in Java, not to be confused with, as we'll see in a bit, static methods in Python).

We use the `@classmethod` decorator to define class methods, and the first argument of these methods will always be the class where the method is defined.

Let's see a simple example first:

In [32]:
class MyClass:
    def hello():
        # this IS intended to be an instance method, we just forgot to add a parameter to capture the instance
        # when this is called from an instance - so this will fail
        print('hello...')
        
    def instance_hello(arg):
        print(f'hello from {arg}')
        
    @classmethod
    def class_hello(arg):
        print(f'hello from {arg}')        

In [36]:
m = MyClass()

In [37]:
m.hello()

TypeError: hello() takes 0 positional arguments but 1 was given

But, as expected, this won't work:

In [38]:
try:
    m.hello()
except TypeError as ex:
    print(ex)

hello() takes 0 positional arguments but 1 was given


On the other hand, notice now the instance method when called from the instance and the class:

In [39]:
hex(id(m))

'0x1561d2a1580'

In [40]:
m.instance_hello()

hello from <__main__.MyClass object at 0x000001561D2A1580>


In [41]:
try:
    MyClass.instance_hello()
except TypeError as ex:
    print(ex)

instance_hello() missing 1 required positional argument: 'arg'


As you can see, the instance method needs to be called from the instance. If we call it from the class, no argument is passed to the function, so we end up with an exception.

This is not the case with class methods - whether we call the method from the class, or the instance, that first argument will always be provided by Python, and will be the class object (not the instance).

Notice how the bindings are different:

In [42]:
MyClass.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

In [43]:
m.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

As you can see in both these cases, `class_hello` is bound to the class.

But with an instance method, the bindings behave differently:

In [44]:
MyClass.instance_hello

<function __main__.MyClass.instance_hello(arg)>

In [45]:
m.instance_hello()
hex(id(m))

hello from <__main__.MyClass object at 0x000001561D2A1580>


'0x1561d2a1580'

So, whenever we call `class_hello` the method is bound to the **class**, and the first argument is the class id:

In [46]:
MyClass.class_hello()

hello from <class '__main__.MyClass'>


In [48]:
m.class_hello

hello from <class '__main__.MyClass'>


In [49]:
m.class_hello()

hello from <class '__main__.MyClass'>


Although in this example I used `arg` as the parameter name in our methods, the normal **convention** is to use `self` and `cls` - that way everyone knows what we're talking about!

We sometimes also want to define functions in a class and always have them be just that - functions, never bound to either the class or the instance, however we call them. Often we do this because we need to utility function that is specific to our class, and we want to keep our class self-contained, or maybe we're writing a library of functions (though modules and packages may be more appropriate for this).

These are called **static** methods. (So be careful here, Python static methods and Java static methods do not have the same meaning!)

We can define static methods using the `@staticmethod` decorator:

In [76]:
class MyClass:
    def instance_hello(self):
        print(f'Instance method bound to {self}')
        
    @classmethod
    def class_hello(cls):
        print(f'Class method bound to {cls}')
        
    @staticmethod
    def static_hello():
        print('Static method not bound to anything')

In [77]:
m = MyClass()
f"id of the instance object m: {hex(id(m))}"

'id of the instance object m: 0x1561d1b5b80'

In [78]:
m.instance_hello()

Instance method bound to <__main__.MyClass object at 0x000001561D1B5B80>


In [80]:
m.instance_hello()
print(f"id of the instance m: {hex(id(m))}")
print(f"id of the function m.instance_hello {hex(id(m.instance_hello))}")

Instance method bound to <__main__.MyClass object at 0x000001561D1B5B80>
id of the instance m: 0x1561d1b5b80
id of the function m.instance_hello 0x1561d1b08c0


In [81]:
print(f"id of MyClass: {hex(id(MyClass))}")
MyClass.class_hello()
print(f"id of MyClass.class_hello: {hex(id(MyClass.class_hello))}")

id of MyClass: 0x1561c760f90
Class method bound to <class '__main__.MyClass'>
id of MyClass.class_hello: 0x1561d1a6b80


In [82]:
m.class_hello()
print(f"id of m.class_hello: {hex(id(m.class_hello))}")

Class method bound to <class '__main__.MyClass'>
id of m.class_hello: 0x1561d1a6ac0


And the static method can be called either from the class or the instance, but is never bound:

In [83]:
MyClass.static_hello
print(f"id of MyClass.static_hello: {hex(id(MyClass.static_hello))}")
print(f"id of MyClass: {hex(id(MyClass))}")

id of MyClass.static_hello: 0x1561d28da60
id of MyClass: 0x1561c760f90


In [84]:
# Why is the ID of static_hello different depending upon on name

m.static_hello
print(f"id of m.static_hello: {hex(id(m.static_hello))}")
m.static_hello
print(f"id of m: {hex(id(m))}")

id of m.static_hello: 0x1561d28da60
id of m: 0x1561d1b5b80


In [85]:
x = MyClass()
x.static_hello
print(f"id of x.static_hello: {hex(id(x.static_hello))}")
x.static_hello
print(f"id of x: {hex(id(x))}")

id of x.static_hello: 0x1561d28da60
id of x: 0x1561d2a15b0


In [89]:
static_hello()

NameError: name 'static_hello' is not defined

In [66]:
MyClass.static_hello()

Static method not bound to anything


In [86]:
hex(id(MyClass.static_hello))

'0x1561d28da60'

In [87]:
m.static_hello()

Static method not bound to anything


In [88]:
hex(id(m.static_hello))

'0x1561d28da60'

#### Example

Let's see a more concrete example of using these different method types.

We're going to create a `Timer` class that will allow us to get the current time (in both UTC and some timezone), as well as record start/stop times.

We want to have the same timezone for all instances of our `Timer` class with an easy way to change the timezone for all instances when needed.

If you need to work with timezones, I recommend you use the `pyrz` 3rd party library. Here, I'll just use the standard library, which is definitely not as easy to use as `pytz`.

In [91]:
from datetime import datetime, timezone, timedelta

class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

So `tz` is a class attribute, and we can set it using a class method `set_timezone` - all instances will share the same `tz` value (unless we override it at the instance level)

In [92]:
Timer.set_tz(-7, 'MST')

In [93]:
Timer.tz

datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')

In [94]:
t1 = Timer()
t2 = Timer()

In [95]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST'),
 datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST'))

In [96]:
Timer.set_tz(-8, 'PST')

In [97]:
t1.tz, t2.tz

(datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST'),
 datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST'))

Next we want a function to return the current UTC time. Obviously this has nothing to do with either the class or the instance, so it is a prime candidate for a static method:

In [98]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)

In [99]:
Timer.current_dt_utc()

datetime.datetime(2022, 3, 1, 3, 31, 28, 968347, tzinfo=datetime.timezone.utc)

In [100]:
t = Timer()

In [101]:
t.current_dt_utc()

datetime.datetime(2022, 3, 1, 3, 32, 9, 575410, tzinfo=datetime.timezone.utc)

Next we want a method that will return the current time based on the set time zone. Obviously the time zone is a class variable, so we'll need to access that, but we don't need any instance data, so this is a prime candidate for a class method:

In [102]:
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    @staticmethod
    def current_dt_utc():
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)

In [103]:
Timer.current_dt_utc(), Timer.current_dt()

(datetime.datetime(2022, 3, 1, 3, 33, 10, 430053, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 3, 1, 3, 33, 10, 430053, tzinfo=datetime.timezone.utc))

In [104]:
t1 = Timer()
t2 = Timer()

In [105]:
t1.current_dt_utc(), t1.current_dt()

(datetime.datetime(2022, 3, 1, 3, 33, 27, 888817, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 3, 1, 3, 33, 27, 888817, tzinfo=datetime.timezone.utc))

In [106]:
t2.current_dt()

datetime.datetime(2022, 3, 1, 3, 33, 29, 961321, tzinfo=datetime.timezone.utc)

And if we change the time zone (we can do so either via the class or the instance, no difference, since the `set_tz` method is always bound to the class):

In [107]:
t2.set_tz(-7, 'MST')

In [108]:
Timer.__dict__

mappingproxy({'__module__': '__main__',
              'tz': datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST'),
              'current_dt_utc': <staticmethod at 0x1561d0f0fa0>,
              'set_tz': <classmethod at 0x1561d1b5b20>,
              'current_dt': <classmethod at 0x1561d1b5250>,
              '__dict__': <attribute '__dict__' of 'Timer' objects>,
              '__weakref__': <attribute '__weakref__' of 'Timer' objects>,
              '__doc__': None})

In [109]:
Timer.current_dt_utc(), Timer.current_dt(), t1.current_dt(), t2.current_dt()

(datetime.datetime(2022, 3, 1, 3, 34, 40, 535509, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 2, 28, 20, 34, 40, 535509, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')),
 datetime.datetime(2022, 2, 28, 20, 34, 40, 535509, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')),
 datetime.datetime(2022, 2, 28, 20, 34, 40, 535509, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), 'MST')))

So far we have not needed any instances to work with this class!

Now we're going to add functionality to start/stop a timer. Obviously we want this to be instance based, since we want to be able to create multiple timers.

In [ ]:
class TimerError(Exception):
    """A custom exception used for Timer class"""
    # (since """...""" is a statement, we don't need to pass)
    
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    def __init__(self):
        # use these instance variables to keep track of start/end times
        self._time_start = None
        self._time_end = None
        
    @staticmethod
    def current_dt_utc():
        """Returns non-naive current UTC"""
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        # internally we always non-naive UTC
        self._time_start = self.current_dt_utc()
        self._time_end = None
        
    def stop(self):
        if self._time_start is None:
            # cannot stop if timer was not started!
            raise TimerError('Timer must be started before it can be stopped.')
        self._time_end = self.current_dt_utc()
        
    @property
    def start_time(self):
        if self._time_start is None:
            raise TimerError('Timer has not been started.')
        # since tz is a class variable, we can just as easily access it from self
        return self._time_start.astimezone(self.tz)  
        
    @property
    def end_time(self):
        if self._time_end is None:
            raise TimerError('Timer has not been stopped.')
        return self._time_end.astimezone(self.tz)
    
    @property
    def elapsed(self):
        if self._time_start is None:
            raise TimerError('Timer must be started before an elapsed time is available')
            
        if self._time_end is None:
            # timer has not ben stopped, calculate elapsed between start and now
            elapsed_time = self.current_dt_utc() - self._time_start
        else:
            # timer has been stopped, calculate elapsed between start and end
            elapsed_time = self._time_end - self._time_start
            
        return elapsed_time.total_seconds()

In [ ]:
from time import sleep

t1 = Timer()
t1.start()
sleep(2)
t1.stop()
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

In [ ]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

So our timer works. Furthermore, we want to use `MST` throughout our application, so we'll set it, and since it's a class level attribute, we only need to change it once:

In [ ]:
Timer.set_tz(-7, 'MST')

In [ ]:
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

In [ ]:
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')